### Indicadores para el Programa de Desarrollo Científico, Tecnológico y de Innovación propuestos por SEDECO

In [2]:
if(! require('pacman')) install.packages('pacman')
pacman::p_load(tidyverse, readxl, janitor, stringr, writexl, foreign, downloader, tools)

Cargando paquete requerido: pacman



In [3]:
#Consideraciones

#Tiempo en línea
options(timeout = 8000)

#Directorio de trabajo
dir.create('Censos Económicos', showWarnings = FALSE)

Cargar datos de Producto Interno Bruto Estatal

Se automatizará la descarga y lectura de datos*

In [4]:
dir.create('Producto Interno Bruto Estatal')

In [5]:
download.file('https://www.inegi.org.mx/contenidos/programas/pibent/2018/tabulados/ori/PIBE_57.xlsx', 
destfile = 'Producto Interno Bruto Estatal/PIBE_57.xlsx', mode = 'wb')

pibe <- read_xlsx('Producto Interno Bruto Estatal/PIBE_57.xlsx', range = 'A5:V511') %>% 
slice(1:46) %>%
clean_names()

print(pibe)

# A tibble: 46 × 22
   concepto    x2003 x2004 x2005 x2006 x2007 x2008 x2009 x2010 x2011 x2012 x2013
   <chr>       <chr> <chr> <chr> <chr> <chr> <chr> <chr> <chr> <chr> <chr> <chr>
 1 Millones d… NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA   
 2 ____aB.1bP… 2842… 2962… 2991… 3134… 3142… 3183… 3008… 3125… 3231… 3332… 3363…
 3 D.21-D.31 … 1876… 1959… 2019… 2100… 2054… 2130… 1955… 1888… 1898… 1897… 1898…
 4 ____aB.1bV… NA    NA    NA    NA    NA    NA    NA    NA    NA    NA    NA   
 5 Actividad … 2655… 2766… 2789… 2924… 2937… 2970… 2813… 2936… 3041… 3142… 3173…
 6 Actividade… 1689… 1646… 1756… 1600… 1521… 1445… 1531… 1524… 1563… 1571… 1402…
 7 11 - Agric… 1689… 1646… 1756… 1600… 1521… 1445… 1531… 1524… 1563… 1571… 1402…
 8 111 - Agri… 1340… 1305… 1360… 1406… 1375… 1283… 1366… 1363… 1406… 1424… 1244…
 9 112 - Cría… 206.… 201.… 167.… 165.… 130.… 157.… 157.… 158.… 154.… 144.… 155.…
10 114 - Pesc… 0     0     0     0     0     0     0     0     0     0     0    
# ℹ 36 m

Cargar datos de la ANUIES

In [6]:
#Definir carpetas
base_dir <- 'ANUIES'

# Carpetas por año que quieres leer
carpetas_anios <- c("2018-2019", "2019-2020", "2020-2021", "2021-2022", "2022-2023", "2023-2024")

In [ ]:
# Obtener el archivo todas_c_... correspondiente dentro de cada carpeta
obtener_archivo_tot <- function(carpeta){
  ruta_carpeta <- file.path(base_dir, carpeta)
  # Buscar archivo que empiece con "Tot_" y contenga el año, ejemplo: todas_c_2018_2019.xlsx
  patron <- paste0("todas_c_", str_replace_all(carpeta, "-", "_"))  # ejemplo: "todas_c_2018_2019"
  
  archivos_en_carpeta <- list.files(ruta_carpeta, pattern = paste0("^", patron, ".*\\.xls[xm]?$"), full.names = TRUE)
  
  if(length(archivos_en_carpeta) == 0){
    warning(paste("No se encontró archivo para carpeta:", carpeta))
    return(NA_character_)
  }
  # Retorna el primer archivo que coincida
  archivos_en_carpeta[1]
}

# Obtener la lista de archivos que quieres leer (uno por carpeta)
archivos_totales <- map_chr(carpetas_anios, obtener_archivo_tot)

# Filtrar archivos existentes (elimina NA)
archivos_totales <- archivos_totales[!is.na(archivos_totales)]

# Leer cada archivo con el rango que quieres y agregar columna con año
lista_anuies <- map2(archivos_totales, carpetas_anios, ~ {
  cat("Leyendo archivo:", .x, "\n")
  read_excel(.x, range = "A4:E14") %>%
    mutate(origen = .y)  # Guardar como origen para identificar el año
})

In [ ]:
#Para poder hacer cálculos, unimos todas las tablas en una sola
anuies <- bind_rows(lista_anuies) %>%
clean_names()

Descargar datos Censos Económicos


In [ ]:

leer_ce <- function(anios, carpeta_destino = "Censos Económicos") {
  # Validar y preparar inputs
  urls <- list(
    '2004' = 'https://www.inegi.org.mx/contenidos/programas/ce/2019/Datosabiertos/ce2004_cdmx_csv.zip',
    '2009' = 'https://www.inegi.org.mx/contenidos/programas/ce/2019/Datosabiertos/ce2009_cdmx_csv.zip',
    '2014' = 'https://www.inegi.org.mx/contenidos/programas/ce/2019/Datosabiertos/ce2014_cdmx_csv.zip',
    '2019' = 'https://www.inegi.org.mx/contenidos/programas/ce/2019/Datosabiertos/ce2019_cdmx_csv.zip'
  )
  
  anios <- as.character(anios)
  
  if (!all(anios %in% names(urls))) {
    stop("Algunos años no son válidos. Usa 2004, 2009, 2014 o 2019")
  }
  
  if (!dir.exists(carpeta_destino)) {
    dir.create(carpeta_destino)
  }
  
  data_list <- list()
  
  for (anio in anios) {
    archivo_zip <- file.path(carpeta_destino, basename(urls[[anio]]))
    
    # Descargar ZIP si no existe
    if (!file.exists(archivo_zip)) {
      download.file(urls[[anio]], archivo_zip, mode = "wb")
    }
    
    # Construir ruta CSV esperado
    nombre_csv <- file.path(carpeta_destino, "conjunto_de_datos", paste0("ce", anio, "_cdmx.csv"))
    
    # Descomprimir sólo si el CSV no existe
    if (!file.exists(nombre_csv)) {
      unzip(archivo_zip, exdir = carpeta_destino)
    }
    
    # Leer y limpiar nombres
    censos <- read_csv(nombre_csv, col_types = NULL) %>% clean_names()
    
    # Guardar en lista con nombre del año
    data_list[[anio]] <- censos
  }
  
  return(data_list)
}


In [ ]:
lista_censos <- leer_ce(c(2004, 2009, 2014, 2019))

ERROR: Error in leer_ce(c(2004, 2009, 2014, 2019)): no se pudo encontrar la función "leer_ce"


### Cálculo de Indicadores

Participación porcentual del PIB de la Industria Química, Hule y Plástico de la Ciudad de México

In [ ]:
str(pibe)

tibble [46 × 22] (S3: tbl_df/tbl/data.frame)
 $ concepto: chr [1:46] "Millones de pesos a precios de 2018" "____aB.1bP - Producto interno bruto" "D.21-D.31 - Impuestos sobre los productos, netos" "____aB.1bV - Valor agregado bruto" ...
 $ x2003   : chr [1:46] NA "2842788.554" "187647.974" NA ...
 $ x2004   : chr [1:46] NA "2962342.214" "195962.04" NA ...
 $ x2005   : chr [1:46] NA "2991075.244" "201941.301" NA ...
 $ x2006   : chr [1:46] NA "3134225.217" "210094.844" NA ...
 $ x2007   : chr [1:46] NA "3142463.801" "205403.358" NA ...
 $ x2008   : chr [1:46] NA "3183429.429" "213092.81" NA ...
 $ x2009   : chr [1:46] NA "3008811.766" "195562.338" NA ...
 $ x2010   : chr [1:46] NA "3125716.691" "188857.545" NA ...
 $ x2011   : chr [1:46] NA "3231440.239" "189868.058" NA ...
 $ x2012   : chr [1:46] NA "3332208.721" "189798.117" NA ...
 $ x2013   : chr [1:46] NA "3363394.325" "189832.939" NA ...
 $ x2014   : chr [1:46] NA "3412110.198" "189637.641" NA ...
 $ x2015   : chr [1:46] NA "350352

In [ ]:
pib_total <- pibe %>%
  filter(str_detect(concepto, regex("Producto interno bruto", ignore_case = TRUE))) %>%
  pivot_longer(
    cols = starts_with("x"),
    names_to = "anio",
    names_prefix = "x",
    values_to = "pib") %>%
  mutate(pib = as.numeric(pib)) %>%
  select(-concepto)    


print(pib_total)


# A tibble: 21 × 2
   anio       pib
   <chr>    <dbl>
 1 2003  2842789.
 2 2004  2962342.
 3 2005  2991075.
 4 2006  3134225.
 5 2007  3142464.
 6 2008  3183429.
 7 2009  3008812.
 8 2010  3125717.
 9 2011  3231440.
10 2012  3332209.
# ℹ 11 more rows


In [ ]:
quimica <- pibe %>%
filter(str_detect(concepto, 'química')) %>%
pivot_longer(cols = starts_with("x"),         # Selecciona columnas desde x2003 hasta x2023p
    names_to = 'anio',                
    values_to = 'aporte_quim',             
    names_prefix = 'x') %>%      # quita la x inicial
mutate(anio = gsub('^x', '', anio), 
  aporte_quim = as.numeric(aporte_quim)) %>%  
select(-concepto)

print(quimica)

# A tibble: 21 × 2
   anio  aporte_quim
   <chr>       <dbl>
 1 2003       82089.
 2 2004       84524.
 3 2005       85285.
 4 2006       87719.
 5 2007       90640.
 6 2008       90861.
 7 2009       83858.
 8 2010       75936.
 9 2011       72114.
10 2012       69233.
# ℹ 11 more rows


In [ ]:
pib_quimica <- pib_total %>%
right_join(quimica, by = 'anio') %>%
mutate(pib_quimica = aporte_quim/pib * 100)

tail(pib_quimica)

anio,pib,aporte_quim,pib_quimica
<chr>,<dbl>,<dbl>,<dbl>
2018,3694575,57790.29,1.564193
2019,3679917,56326.91,1.530657
2020,3293855,54150.02,1.643971
2021,3496056,59144.79,1.691758
2022r,3650609,59039.73,1.617257
2023p,3806083,56953.39,1.496378


Participación porcentual del PIB de los Servicios Profesionales, Científicos y Técnicos de la Ciudad de México


In [ ]:
cientificos <- pibe %>%
filter(str_detect(concepto, '54 - Servicios profesionales, científicos y técnicos')) %>%
pivot_longer(cols = starts_with("x"),         # Selecciona columnas desde x2003 hasta x2023p
    names_to = 'anio',                # Nuevo nombre para la columna de años
    values_to = 'aporte_cienti',             # Nuevo nombre para los valores
    names_prefix = 'x') %>%      # quita la x inicial
mutate(anio = gsub('^x', '', anio), # Elimina la "x" inicial para que año quede limpio
  aporte_cienti = as.numeric(aporte_cienti)) %>%  
select(-concepto)

print(cientificos)

# A tibble: 21 × 2
   anio  aporte_cienti
   <chr>         <dbl>
 1 2003        164642.
 2 2004        167701.
 3 2005        171250.
 4 2006        176419.
 5 2007        179809.
 6 2008        177666.
 7 2009        175442.
 8 2010        177437.
 9 2011        179657.
10 2012        178935.
# ℹ 11 more rows


In [ ]:
pib_ciencia <- pib_total %>%
right_join(cientificos, by = 'anio') %>%
mutate(pib_ciencia = aporte_cienti/pib * 100)

tail(pib_ciencia)

anio,pib,aporte_cienti,pib_ciencia
<chr>,<dbl>,<dbl>,<dbl>
2018,3694575,189600.1,5.131851
2019,3679917,193165.1,5.249169
2020,3293855,174481.3,5.297176
2021,3496056,179685.5,5.139663
2022r,3650609,203447.8,5.572981
2023p,3806083,215707.0,5.667428


Inversión total en los servicios profesionales, científicos y técnicos

In [ ]:
inversiones_totales <- map_dfr(
    names(lista_censos),
    function(anio) {
        df <- lista_censos[[anio]] %>%
        mutate(codigo = as.numeric(codigo))

        inver_gen <- df %>%
        filter(codigo == 54, is.na(id_estrato)) %>%
        slice(1) %>%
        pull(a211a)

        inver_espec <- df  %>%
        filter(codigo == 5417, is.na(id_estrato)) %>%
        slice(1) %>%
        pull(a211a)
        
        tibble(anio = as.integer(anio),
        gran_sector = round(inver_gen, 3),
        sector = round(inver_espec, 3) )

    }
)

head(inversiones_totales)

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"


anio,gran_sector,sector
<int>,<dbl>,<dbl>
2004,789.488,12.720
2009,1636.274,23.701
2014,1171.968,13.671
2019,1915.817,25.549


Participación porcentual Total del personal ocupado en los Servicios profesionales, científicos y técnicos 

In [ ]:
ocupacion_totales <- map_dfr(
  names(lista_censos),
  function(anio) {
    df <- lista_censos[[anio]] %>%
      mutate(codigo = as.numeric(codigo))

    ocu_total <- df %>%
      select(codigo, h001a) %>%
      slice(1) %>%
      pull(h001a)

    ocu_gran_sector <- df %>%
      filter(codigo == 54 & !is.na(h001a)) %>%
      slice(1) %>%
      pull(h001a)

    ocu_sector <- df %>%
      filter(codigo == 5417 & !is.na(h001a)) %>%
      slice(1) %>%
      pull(h001a)


    porcentaje <- (ocu_sector / ocu_total) * 100

    tibble(
      anio = as.integer(anio),
      ocupacion_total = round(ocu_total, 3),
      ocupacion_gran_sector = round(ocu_gran_sector, 3),
      ocupacion_sector = round(ocu_sector, 3),
      porcentaje = porcentaje
    )
  }
)
head(ocupacion_totales)

Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"
Warning message:
"There was 1 warning in `mutate()`.
ℹ In argument: `codigo = as.numeric(codigo)`.
Caused by warning:
! NAs introducidos por coerción"


anio,ocupacion_total,ocupacion_gran_sector,ocupacion_sector,porcentaje
<int>,<dbl>,<dbl>,<dbl>,<dbl>
2004,2842874,179503,1413,0.04970322
2009,3299325,187312,1115,0.03379479
2014,3603572,208693,2124,0.05894152
2019,4297134,286407,4176,0.09718105


Porcentaje de egresados con conocimientos en Ciencia, Tecnología e Innovación

-Disciplinas en ciencia, tecnología, ingeniería y matemáticas.

In [ ]:
str(anuies)

tibble [60 × 6] (S3: tbl_df/tbl/data.frame)
 $ entidad_federativa       : chr [1:60] "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" ...
 $ campo_amplio_de_formacion: chr [1:60] "Administración y negocios" "Agronomía y veterinaria" "Artes y humanidades" "Ciencias de la salud" ...
 $ egresados_mujeres        : num [1:60] 12593 541 3349 8107 2157 ...
 $ egresados_hombres        : num [1:60] 11944 296 2579 4016 2224 ...
 $ egresados_total          : num [1:60] 24537 837 5928 12123 4381 ...
 $ origen                   : chr [1:60] "2018-2019" "2018-2019" "2018-2019" "2018-2019" ...


In [ ]:
#Total de egresados por periodo
total_carreras <- anuies %>%
group_by(origen) %>%
summarise(total_carreras = sum (egresados_total, na.rm = TRUE)) %>%
arrange(origen)

total_carreras

origen,total_carreras
<chr>,<dbl>
2018-2019,102103
2019-2020,105647
2020-2021,118117
2021-2022,134181
2022-2023,147419
2023-2024,153746


In [ ]:
# Obtener el archivo STEM... correspondiente dentro de cada carpeta
pob_stem <- function(carpeta){
  ruta_carpeta <- file.path(base_dir, carpeta)
  # Buscar archivo que empiece con "STEM_" y contenga el año, ejemplo: STEM_2018_2019.xlsx
  patron <- paste0("STEM_", str_replace_all(carpeta, "-", "_"))  # ej: "STEM_2018_2019"
  
  archivos_en_carpeta <- list.files(ruta_carpeta, pattern = paste0("^", patron, ".*\\.xls[xm]?$"), full.names = TRUE)
  
  if(length(archivos_en_carpeta) == 0){
    warning(paste("No se encontró archivo para carpeta:", carpeta))
    return(NA_character_)
  }
  # Retorna el primer archivo que coincida
  archivos_en_carpeta[1]
}

# Obtener la lista de archivos que quieres leer (uno por carpeta)
stem_tot <- map_chr(carpetas_anios, pob_stem)

# Filtrar archivos existentes (elimina NA)
stem_tot <- stem_tot[!is.na(stem_tot)]

# Leer cada archivo con el rango que quieres y agregar columna con año
lista_tablas_stem <- map2(stem_tot, carpetas_anios, ~ {
  cat("Leyendo archivo:", .x, "\n")
  read_excel(.x, range = "A4:E8") %>%
    mutate(origen = .y)  # Guardar como origen para identificar el año
})

Leyendo archivo: ANUIES/2018-2019/STEM_2018_2019.xlsx 
Leyendo archivo: ANUIES/2019-2020/STEM_2019_2020.xlsx 
Leyendo archivo: ANUIES/2020-2021/STEM_2020_2021.xlsx 
Leyendo archivo: ANUIES/2021-2022/STEM_2021_2022.xlsx 
Leyendo archivo: ANUIES/2022-2023/STEM_2022_2023.xlsx 
Leyendo archivo: ANUIES/2023-2024/STEM_2023_2024.xlsx 


In [ ]:
total_stem <- bind_rows(lista_tablas_stem) %>%
clean_names()

str(total_stem)

tibble [24 × 6] (S3: tbl_df/tbl/data.frame)
 $ entidad_federativa       : chr [1:24] "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" ...
 $ campo_amplio_de_formacion: chr [1:24] "Ciencias de la salud" "Ciencias naturales, matemáticas y estadística" "Ingeniería, manufactura y construcción" "Tecnologías de la información y la comunicación" ...
 $ egresados_mujeres        : num [1:24] 8107 2157 4532 1352 7788 ...
 $ egresados_hombres        : num [1:24] 4016 2224 9011 4400 3752 ...
 $ egresados_total          : num [1:24] 12123 4381 13543 5752 11540 ...
 $ origen                   : chr [1:24] "2018-2019" "2018-2019" "2018-2019" "2018-2019" ...


In [ ]:
#Total de egresados con conocimientos STEM por periodo
egresados_stem <- total_stem %>%
group_by(origen) %>%
summarise(total_egresados = sum (egresados_total, na.rm = TRUE)) %>%
arrange(origen)

print(egresados_stem)

# A tibble: 6 × 2
  origen    total_egresados
  <chr>               <dbl>
1 2018-2019           35799
2 2019-2020           35242
3 2020-2021           42898
4 2021-2022           44086
5 2022-2023           48201
6 2023-2024           46586


In [ ]:
#Porcentaje de egresados STEM = (Total de egresados STEM / Total de egresados TODAS LAS CARRERAS) * 100
porcentaje_cti <- total_carreras %>%
left_join(egresados_stem, by = 'origen') %>%
mutate(porcentaje = (total_egresados / total_carreras) * 100)

print(porcentaje_cti)

# A tibble: 6 × 4
  origen    total_carreras total_egresados porcentaje
  <chr>              <dbl>           <dbl>      <dbl>
1 2018-2019         102103           35799       35.1
2 2019-2020         105647           35242       33.4
3 2020-2021         118117           42898       36.3
4 2021-2022         134181           44086       32.9
5 2022-2023         147419           48201       32.7
6 2023-2024         153746           46586       30.3


Porcentaje de mujeres egresadas con conocimientos en Ciencia, Tecnología e Innovación

(Mujeres egresadas / Mujeres egresadas de todas las disciplinas) * 100


In [ ]:
str(lista_anuies)

List of 6
 $ : tibble [10 × 6] (S3: tbl_df/tbl/data.frame)
  ..$ ENTIDAD FEDERATIVA       : chr [1:10] "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" ...
  ..$ CAMPO AMPLIO DE FORMACIÓN: chr [1:10] "Administración y negocios" "Agronomía y veterinaria" "Artes y humanidades" "Ciencias de la salud" ...
  ..$ Egresados Mujeres        : num [1:10] 12593 541 3349 8107 2157 ...
  ..$ Egresados Hombres        : num [1:10] 11944 296 2579 4016 2224 ...
  ..$ Egresados Total          : num [1:10] 24537 837 5928 12123 4381 ...
  ..$ origen                   : chr [1:10] "2018-2019" "2018-2019" "2018-2019" "2018-2019" ...
 $ : tibble [10 × 6] (S3: tbl_df/tbl/data.frame)
  ..$ ENTIDAD FEDERATIVA       : chr [1:10] "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" "CIUDAD DE MÉXICO" ...
  ..$ CAMPO AMPLIO DE FORMACIÓN: chr [1:10] "Administración y negocios" "Agronomía y veterinaria" "Artes y humanidades" "Ciencias de la salud" ...
  ..$ Egresados Mujeres        : n

In [ ]:
#Mujeres egresadas de todas las carreras
total_mujeres <- map_dfr(
lista_anuies, ~ .x %>%
group_by(origen) %>%
summarise(total_mujeres = sum(`Egresados Mujeres`, na.rm = TRUE))) %>%
arrange(origen)

total_mujeres

origen,total_mujeres
<chr>,<dbl>
2018-2019,53885
2019-2020,56112
2020-2021,63217
2021-2022,72831
2022-2023,79401
2023-2024,84213


In [ ]:
#MUJERES EGRESADAS STEM
mujeres_stem <- total_stem %>%
group_by(origen) %>%
summarise(mujeres_stem = sum(egresados_mujeres, na.rm = TRUE)) %>%
arrange(origen)

mujeres_stem

origen,mujeres_stem
<chr>,<dbl>
2018-2019,16148
2019-2020,15523
2020-2021,19737
2021-2022,20042
2022-2023,22380
2023-2024,21577


In [ ]:
mujeres_stem <- mujeres_stem %>%
left_join(total_mujeres, by = 'origen') %>%
mutate(porcentaje_mujeres_stem = (mujeres_stem / total_mujeres) * 100)

mujeres_stem

origen,mujeres_stem,total_mujeres,porcentaje_mujeres_stem
<chr>,<dbl>,<dbl>,<dbl>
2018-2019,16148,53885,29.96752
2019-2020,15523,56112,27.66431
2020-2021,19737,63217,31.22103
2021-2022,20042,72831,27.51850
2022-2023,22380,79401,28.18604
2023-2024,21577,84213,25.62193


In [ ]:
#Guardar los resultados en excel
write_xlsx(list(Aporte_química = pib_quimica, Aporte_Serv_Prof = pib_ciencia, Inversión_total_Serv_Prof = inversiones_totales, Personal_Ocup_Serv_Prof = ocupacion_totales, Egresados_STEM = porcentaje_cti, Egresadas_Mujeres_STEM = mujeres_stem), path = 'Resultados/Indicadores_CTI.xlsx')